# Using PyTorch

In [3]:
import torch 
import torch.nn as nn 
import numpy as np 
import torchinfo 
from torchinfo import summary 
from tqdm import tqdm
import math

def equation(input:torch.Tensor): # z = 5 * a^2 + 3 * b^2
    return torch.matmul(torch.mul(input, input), torch.tensor([5, 3], dtype=torch.float32))

num_of_data = 100000
inputs = torch.randn(num_of_data, 2)
outputs = equation(inputs)

class SimpleModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 10)
        self.fc2 = nn.Linear(10, 1)
        self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        return x 

device = torch.device('mps')
split = int(inputs.shape[0] * 0.7)
X_train, y_train, X_test, y_test = inputs[:split].to(device), outputs[:split].to(device), inputs[split:].to(device), outputs[split:].to(device)
print(f"Shape of X_train: {X_train.shape}, y_train: {y_train.shape}, X_test: {X_test.shape}, y_test = {y_test.shape}")

model = SimpleModel().to(device)
summary(model) 

Shape of X_train: torch.Size([70000, 2]), y_train: torch.Size([70000]), X_test: torch.Size([30000, 2]), y_test = torch.Size([30000])


Layer (type:depth-idx)                   Param #
SimpleModel                              --
├─Linear: 1-1                            30
├─Linear: 1-2                            11
├─Sigmoid: 1-3                           --
Total params: 41
Trainable params: 41
Non-trainable params: 0

In [4]:
lr = 0.01
n_epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr = lr)
criterion = nn.MSELoss()
batch_size = 100
log_interval = 1

for epoch in range(n_epochs):
    print(f"====Epoch {epoch}====")
    training_loss = 0
    training_count = 0
    for batch_end_idx in tqdm(range(batch_size, X_train.shape[0], batch_size)):
        optimizer.zero_grad()
        out = model(X_train[:batch_end_idx])
        loss = criterion(out, y_train[:batch_end_idx].view(-1,1))
        loss.backward()
        optimizer.step()
        training_loss += loss.item()
        training_count += 1

    if epoch % log_interval == 0:
        model.eval()
        validation_loss = 0
        validation_count = 0
        for batch_end_idx in range(batch_size, X_test.shape[0], batch_size):
            out = model(X_test[:batch_end_idx])
            loss = criterion(out, y_test[:batch_end_idx].view(-1,1))
            validation_loss += loss.item()
            validation_count += 1
        print(f"Training Loss: {training_loss / training_count} | Validation Loss: {validation_loss / validation_count}")
        model.train()

====Epoch 0====


100%|██████████| 699/699 [00:11<00:00, 59.39it/s]


Training Loss: 58.7142123930444 | Validation Loss: 36.887394142788786
====Epoch 1====


100%|██████████| 699/699 [00:10<00:00, 69.48it/s] 


Training Loss: 22.56092092953356 | Validation Loss: 13.007554610836067
====Epoch 2====


...

Training Loss: 3.339565985874727 | Validation Loss: 3.2308849148128345
====Epoch 8====


100%|██████████| 699/699 [00:10<00:00, 68.45it/s] 


Training Loss: 3.029321526935343 | Validation Loss: 2.953049585372708
====Epoch 9====


100%|██████████| 699/699 [00:09<00:00, 70.96it/s] 


Training Loss: 2.7777490767627655 | Validation Loss: 2.7245000437350577
